<a href="https://colab.research.google.com/github/JRcepeda/Preprocesamiento_y_modelos_R/blob/main/M%C3%A9todo_de_ensamble_en_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('randomForest')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages('caret')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages('Metrics')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(Metrics)

In [ ]:
library(caret)

In [ ]:
require(randomForest)

In [ ]:
datos<-read.csv("/content/datos_s (1).csv")

In [ ]:
dim(datos)

[1] 2261   14

In [ ]:
head(datos)

title                                  artist top.genre year bpm energy
1 Flowers                                0      0         2023 118 68    
2 Cupid - Twin Ver.                      1      1         2023 120 59    
3 BESO                                   2      0         2023  95 64    
4 Boy's a liar Pt. 2                     3      2         2023 133 81    
5 Creepin' (with The Weeknd & 21 Savage) 4      3         2022  98 62    
6 Daylight                               5      4         2023 130 43    
  danceability dB liveness valence duration acousticness speechiness popularity
1 71           -4  3       65      200       6            7          98        
2 78           -8 35       73      174      44            3          97        
3 77           -7 17       53      195      74           14          96        
4 70           -8 25       86      131      25            5          96        
5 72           -6  8       17      222      42            5          96        
6 51           -9  9       32      213      83            3          96

In [ ]:
anyDuplicated(datos$title)

[1] 0

In [ ]:
datos=datos[!duplicated(datos$title),]

In [ ]:
dim(datos)

[1] 2261   13

In [ ]:
titulo=datos$title

In [ ]:
rownames(datos)=datos$title

In [ ]:
datos=datos[,-which(names(datos)=='title')]

In [ ]:
print(sapply(datos,class))

      artist    top.genre         year          bpm       energy danceability 
   "integer"    "integer"    "integer"    "integer"    "integer"    "integer" 
          dB     liveness      valence     duration acousticness  speechiness 
   "integer"    "integer"    "integer"    "integer"    "integer"    "integer" 
  popularity 
   "integer" 


In [ ]:
#datos$artist=as.integer(as.factor(datos$artist))-1

In [ ]:
#datos$top.genre=as.integer(as.factor(datos$top.genre))-1

In [ ]:
set.seed(12)
traini=createDataPartition(datos$popularity,p=0.8,list=FALSE)

In [ ]:
test=datos[-traini,]

In [ ]:
train=datos[traini,]

In [ ]:
sum(is.na(datos))

[1] 0

In [ ]:
print(sapply(datos,class))

      artist    top.genre         year          bpm       energy danceability 
   "integer"    "integer"    "integer"    "integer"    "integer"    "integer" 
          dB     liveness      valence     duration acousticness  speechiness 
   "integer"    "integer"    "integer"    "integer"    "integer"    "integer" 
  popularity 
   "integer" 


#Bosque aleatorio 1:

Estimadores 250 y profundidad 50

In [ ]:
bosque1=randomForest(popularity~.,data=train,ntree=250,nodesize=2,splitrule="mse",maxdepth=50,importance = TRUE)

In [ ]:
PredB1=predict(bosque1,test)

In [ ]:
r2_score<-function(test,pred){
  meanYt=mean(test$popularity)
  N=sum(((test$popularity-pred)**2))
  D=sum(((test$popularity-meanYt)**2))
  r2=1-(N/D)
  return (r2)
}

In [ ]:
MetricB1=r2_score(test,PredB1)

#Bosque aleatorio 2:

Estimadores 250 y profundidad 150

In [ ]:
bosque2=randomForest(popularity~.,ntree=250,maxdepth=150,splitrule='mse',data=train)

In [ ]:
PredB2=predict(bosque2,test)

In [ ]:
MetricB2=r2_score(test,PredB2)

#Bosque aleatorio 3:

Estimadores 500 y profundidad 50

In [ ]:
bosque3=randomForest(popularity~.,ntree=500,maxdepth=50,splitrule='mse',data=train)

In [ ]:
PredB3=predict(bosque3,test)

In [ ]:
MetricB3=r2_score(test,PredB3)

#Bosque aleatorio 4:

Estimadores 500 y profundidad 150

In [ ]:
bosque4=randomForest(popularity~.,ntree=500,maxdepth=150,splitrule='mse',data=train)

In [ ]:
PredB4=predict(bosque4,test)

In [ ]:
MetricB4=r2_score(test,PredB4)

#Resultados

In [ ]:
Modelo=c('Bosque 1  Estimadores 250 Profundidad 50','Bosque 2  Estimadores 250 Profundidad 150','Bosque 3  Estimadores 500 Profundidad 50','Bosque 4  Estimadores 500 Profundidad 150')

In [ ]:
R2=c(MetricB1,MetricB2,MetricB3,MetricB4)

In [ ]:
resultados=data.frame(Modelo,R2)

In [ ]:
resultados

Modelo                                    R2       
1 Bosque 1  Estimadores 250 Profundidad 50  0.6818921
2 Bosque 2  Estimadores 250 Profundidad 150 0.6740856
3 Bosque 3  Estimadores 500 Profundidad 50  0.6797404
4 Bosque 4  Estimadores 500 Profundidad 150 0.6818140

In [ ]:
bosque1$importance

%IncMSE     IncNodePurity
artist       119.8549199 105287.737   
top.genre     11.4080467  22222.320   
year          18.7810601  24301.765   
bpm            0.1739223   9559.111   
energy         4.2770938   8846.653   
danceability   1.3955506   9054.562   
dB             1.3866674   5022.627   
liveness       0.3755932   7917.464   
valence        0.8030673   8781.180   
duration       1.6701826  11057.720   
acousticness   0.7840511   7637.180   
speechiness    0.5926807   6743.586